# Objetivo

Objetivo é avaliar as sugestões obtidas, comparando-as com uma parte separada do portfólio, além de verificar se o resultado é melhor que uma escolha aleatória.

# Método

Neste problema de recomendação, não temos uma variável-alvo definida, nem uma base explícita de teste.

Então, para fazer a avaliação dos resultados, separamos o portfólio em duas partes: uma (`X_train`) é usada para gerar a lista de sugestões. A outra (`X_test`) é a base em que procuramos nossas sugestões. Idealmente, todos os IDs que estão em `X_test` deveriam estar na lista de sugestões.

Definimos uma função para contar o número de **hits**, ou seja, quantos itens sugeridos estão dentro da parte do portfólio separada para teste. 

Por fim, comparamos o número de _hits_ com uma escolha aleatória na base completa. Usamos uma simulação de Monte Carlo, em que o mesmo número de sugestões é aleatoriamente obtido da base completa _market_, usando o método `sample` do Pandas, sendo esse processo repetido várias vezes (`monte_carlo_size`), para obter uma estimativa mais robusta.

# Preparação (Imports)

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# preprocessing
from sklearn import preprocessing

#Unsupervised Learning
from sklearn.neighbors import NearestNeighbors

#Imports para Visualizações
import folium
from folium import plugins
import random

# Leitura do arquivo de entrada (portfólio com IDs de clientes atuais)

In [2]:
port = pd.read_csv('estaticos_portfolio3.csv',usecols = ['id'])
port.set_index('id',inplace=True)

# Carregamento da Base de dados (Market)

In [3]:
import os, wget
market_file = 'estaticos_market.feather'
url_market = 'https://owncloud.ifsc.edu.br/index.php/s/vH4JMIAOuAqvmRc/download'
if not os.path.exists(market_file):
    try:
        #Tip from https://stackabuse.com/download-files-with-python/
        print('Downloading market database...')
        wget.download(url_market, market_file)
        print('Done!')
    except:
        print('Error downloading market database. Check url and connection.')
        raise

print('Opening market database...')
base = pd.read_feather(market_file)
print('Done opening market database.')
base.set_index('id',inplace=True)
base.head(2)

Opening market database...
Done opening market database.


,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,...,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,idade_media_socios,qt_socios_feminino,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux
id,,,,,,,,,,,,,,,,,,,,,
a6984c3ae395090e3bee8ad63c3758b110de096d5d819583a784a113726db849,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,14.457534,10 a 15,False,False,...,LESTE POTIGUAR,NATAL,True,2.0,44.0,NaN,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",3132172.8,3132172.8
6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c47fdef50b33f4f56f05,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,SERVICOS DE ALOJAMENTO/ALIMENTACAO,SERVIÇO,1.463014,1 a 5,False,False,...,CENTRO NORTE PIAUIENSE,TERESINA,True,1.0,27.0,NaN,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",210000.0,210000.0


# Pré-Processamento


## Inicialmente identificar como serão preenchidos os NaNs de acordo com seu tipo

In [4]:
fill_dict = base.dtypes.to_dict()
# Categorias em que existiam NaNs e são categóricas, mas deveriam ser booleans
cat2bool = ['fl_optante_simei','fl_optante_simples','fl_passivel_iss']

for feature in fill_dict:
    if(fill_dict[feature] == bool):
        fill_dict[feature] = False
    elif(fill_dict[feature] == object):
        fill_dict[feature] = 'other'
    else:
        fill_dict[feature] = 0
    if feature in cat2bool:
        fill_dict[feature] = False
        
#fill_dict

## Executando o preenchimento de NaNs

In [5]:
base.fillna(value=fill_dict, inplace=True)
base[cat2bool] = base[cat2bool].astype(bool)
bool2numeric = base.columns[base.dtypes == bool]

base[bool2numeric] = base[bool2numeric].astype(int)

## Encoding

In [6]:
cat = base.columns[base.dtypes == object]
not_cat = base.columns[base.dtypes != object]

In [7]:
# Usando LabelEncoder
#https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/

# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [8]:
base_le = pd.DataFrame()
base_le[cat] = base[cat].apply(lambda col: le.fit_transform(col))
base_encoded = pd.concat([base[not_cat],base_le], axis =1)
base_encoded.head()


,fl_matriz,idade_empresa_anos,fl_me,fl_sa,fl_mei,fl_ltda,fl_st_especial,fl_email,fl_telefone,fl_optante_simples,...,nm_divisao,nm_segmento,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo
id,,,,,,,,,,,,,,,,,,,,,
a6984c3ae395090e3bee8ad63c3758b110de096d5d819583a784a113726db849,1,14.457534,0,0,0,0,0,1,1,1,...,32,10,19,4,0,0,6,48,2,2
6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c47fdef50b33f4f56f05,1,1.463014,0,0,1,0,0,1,1,0,...,3,3,16,2,1,1,4,69,10,10
4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02edc5147bc2a2535b08,1,7.093151,0,0,1,0,0,0,1,1,...,86,20,2,0,0,2,2,43,1,1
3348900fe63216a439d2e5238c79ddd46ede454df7b9d8c24ac33eb21d4b21ef,1,6.512329,0,0,0,0,0,1,1,1,...,74,17,2,0,0,2,2,43,10,10
1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4b102c769f7611ab29f,1,3.200000,0,0,0,0,0,1,1,1,...,77,8,19,4,0,0,6,48,10,10


Aplicando encoding ao portfólio.

In [9]:
port_le = base_encoded.merge(port, how='right',left_index=True,right_index=True)
port = base.merge(port, how='right',left_index=True,right_index=True)

# Modelo com Nearest Neighbors 

Como não temos variável-alvo, usamos o treinamento não supervisionado do Nearest Neighbors.

In [10]:
%%time

qtd_neighbors = 5
nearest = NearestNeighbors(n_neighbors=qtd_neighbors, metric = 'cosine')

nearest.fit(base_encoded)


CPU times: user 434 ms, sys: 429 ms, total: 863 ms
Wall time: 862 ms


NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

## Separação "treinamento" e teste

Esta separação deve ser usada somente para avaliação, não para a versão final.

In [11]:
from sklearn.model_selection import train_test_split
# ignorar y. Só usamos X
X_train, X_test, y_train, y_test = train_test_split(
    port_le, list(range(len(port_le))), test_size=0.3, random_state=17)
print(f'Tamanho do "treinamento": {X_train.shape}')
print(f'Tamanho do teste para avaliação: {X_test.shape}')

Tamanho do "treinamento": (185, 37)
Tamanho do teste para avaliação: (80, 37)


## Procurando por sugestões

In [12]:
%%time
neighbors_list = {}

for row in range(X_train.shape[0]):
    #print(row)
    neighbors_list[row] = nearest.kneighbors(X_train.iloc[[row]].values)

CPU times: user 2min 21s, sys: 1min 7s, total: 3min 29s
Wall time: 2min 15s


A resposta é uma lista de distâncias e de sugestões de clientes.

# Processamento da lista de sugestões

Da lista de vizinhos, filtrar os clientes pertencentes ao portfólio de entrada.

In [13]:
list_size = len(neighbors_list)
num_neighbors = len(neighbors_list[0][1][0])

neighbors_idx_array = neighbors_list[0][1][0]
neighbors_distance_array = neighbors_list[0][0][0]
np.delete(neighbors_idx_array, [0,1])
np.delete(neighbors_distance_array, [0,1])
for line in range(1,list_size):
    neighbors_idx_array = np.concatenate((neighbors_idx_array, neighbors_list[line][1][0]), axis=None) 
    neighbors_distance_array = np.concatenate((neighbors_distance_array, neighbors_list[line][0][0]), axis=None) 

if len(neighbors_idx_array) != list_size*num_neighbors:
    print("ERROR: Check array size.")
# Temos agora um array unidimensional com os índices dos clientes recomendados)

Criar um dicionário com o id de origem da requisição, id do vizinho e distância entre eles

In [14]:
dicio = {}
for idx,ind in zip(neighbors_idx_array, range(len(neighbors_idx_array))):
    dicio[ind] = (port.iloc[int(ind/qtd_neighbors)].name, base.iloc[idx].name, (neighbors_distance_array[ind]))

Criando um DataFrame com as sugestões.

In [15]:
neig_df = pd.DataFrame.from_dict(dicio,orient='index')
neig_df.rename(columns={0:'id_origin', 1:'id',2:'distance'},inplace = True)
neig_df.set_index('id', inplace=True)

Identificando os conflitos e removendo da lista de sugestões.

In [16]:
conflicts = neig_df.merge(X_train, how='inner',left_index=True,right_index=True)
suggestion_with_conflicts = neig_df.merge(base, how='left',left_index=True,right_index=True)

suggestions = suggestion_with_conflicts.drop(conflicts.index)

**Resultado**: lista de sugestões em `suggestions`.

# Análise dos resultados

In [17]:
# Verificar número de hits (quantas sugestões são parte de um conjunto de teste)
def count_hits_in_suggestion(df_suggestions, df_X_test): #suggestions e X_test são dataframes
    X_list = df_X_test.index.to_list()
    hits = 0
    for sugg in df_suggestions.index.to_list():
        hits = hits + X_list.count(sugg)
    return hits

In [18]:
our_hits = count_hits_in_suggestion(suggestions, X_test)

In [19]:
# Compara com sorteio aleatório (random picking) da base market

n_picking = len(suggestions) # quantidade de sugestões a sortear

# Simulação de Monte Carlo, ou seja, sortear um conjunto várias vezes e depois fazer a média
monte_carlo_size = 500
num_hits = int(0)
for k in range(monte_carlo_size):
    random_suggestion = base_encoded.sample(n = n_picking)
    num_hits = num_hits + count_hits_in_suggestion(random_suggestion, X_test)
mean_num_hits = num_hits / monte_carlo_size

In [20]:
print(f'Number of hits in our suggestion: {our_hits}.')
print(f'They represent {100*our_hits/len(X_test):.2f}% of the test size.\n')

print(f'For random suggestion, we found an average of {mean_num_hits:.2f} hits.')
print(f'They represent {100*mean_num_hits/len(X_test):.2f}% of the test size.')

Number of hits in our suggestion: 0.
They represent 0.00% of the test size.

For random suggestion, we found an average of 0.09 hits.
They represent 0.11% of the test size.
